In [1]:
from dataset import Dataset
from botr import BOTR_Layer, BOTR
from config import GENERATOR_CONFIG_DEFAULT

In [2]:
dataset = Dataset()

=> loading coco asset: dataset/coco-safe-licenses.pickle
=> loading coco object - dataset/annotations/captions_train2017.json
loading annotations into memory...
Done (t=1.05s)
creating index...
index created!


In [4]:
botr = BOTR(GENERATOR_CONFIG_DEFAULT, dataset)

In [ ]:
b